# 1 Data preprocessing

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

Data can be downloaded from https://drive.google.com/drive/folders/1GOhMd0UbGnod2Kgtyw5k8_3KYFlEBHT0?usp=sharing

In [2]:
OD = pd.read_csv("Data/accessibility/ODcost_final/OD_cost_final.csv", encoding = "gbk")

/tmp/ipykernel_5792/957129089.py:1: DtypeWarning: Columns (37,42,45,49,54,56) have mixed types. Specify dtype option on import or set low_memory=False.
  OD = pd.read_csv("Data/accessibility/ODcost_final/OD_cost_final.csv", encoding = "gbk")


In [3]:
OD.head(1)

,Index,IN_FID,NEAR_FID,NEAR_DIST,NEAR_RANK,序号,机构名,机构地,行政区,机构类,...,Rj.1,Unnamed: 48,NEAR_FID.2,求和项:Gij*Rj2,Unnamed: 51,Unnamed: 52,Unnamed: 53,△T.1,Count,Percentage
0,1,0,411,1649.255304,2,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,...,0.000097,NaN,20,0.000006,NaN,0.000002,NaN,>=0,2299.0,10.11%


In [4]:
OD.columns.to_list()

['Index',
 'IN_FID',
 'NEAR_FID',
 'NEAR_DIST',
 'NEAR_RANK',
 '序号',
 '机构名',
 '机构地',
 '行政区',
 '机构类',
 '机构级',
 '经营性',
 '床位数',
 '牙椅数',
 'wgs_lng',
 'wgs_lat',
 'POP',
 'lng_84',
 'lon_84',
 'gcj_lng_D',
 'gcj_lat_D',
 'gcj_lng_O',
 'gcj_lat_O',
 'Distance',
 'Time',
 'Cost',
 'Distance_5',
 'Time_5',
 'Cost_5',
 '△T',
 '△C',
 'Gij',
 'Gij*Dk',
 'Rj1',
 'Gij*Rj1',
 'Rj2',
 'Gij*Rj2',
 'IN_FID.1',
 '求和Gij*Dk',
 'Sj',
 'Rj',
 'Unnamed: 41',
 'NEAR_FID.1',
 '求和项:Gij*Rj',
 'Unnamed: 44',
 'IN_FID.2',
 '求和项:Gij*Dk',
 'Rj.1',
 'Unnamed: 48',
 'NEAR_FID.2',
 '求和项:Gij*Rj2',
 'Unnamed: 51',
 'Unnamed: 52',
 'Unnamed: 53',
 '△T.1',
 'Count',
 'Percentage']

In [5]:
# Reduce unnecessary columns
OD = OD[['Index', 'IN_FID', 'NEAR_FID', '机构名', '床位数', 'wgs_lng', 'wgs_lat', 'POP', 'lng_84', 'lon_84', 'Time', 'Cost','Time_5',
 'Cost_5']]

In [6]:
# Adjust the number of beds (per thousand -- > per person)
OD['床位数'] = OD['床位数'] * 1000

In [7]:
# Check
OD.head(1)

,Index,IN_FID,NEAR_FID,机构名,床位数,wgs_lng,wgs_lat,POP,lng_84,lon_84,Time,Cost,Time_5,Cost_5
0,1,0,411,广州市第一人民医院,990000,113.252797,23.133996,120239.8201,113.268876,23.133263,2042,2,1974,2


# 2 Define Functions

In [8]:
## Gaussian decay function
def get_G(x, t): # x is the cost and t is the cost threshold
    G = (np.exp(-0.5 * (x/t)**2) - np.exp(-0.5)) / (1 - np.exp(-0.5))
    return G

## G*Dk
def get_1(df, c, t): # df is the data table to be calculated, c is the cost column name to be calculated, and t is the cost threshold
    # Calculates and saves column G
    df['G'] = df[c].apply(lambda x: get_G(x, t))
    
    # Calculate the G*Dk columns, using the already calculated G columns
    df['G*Dk'] = df['POP'] * df['G']

    return df

In [9]:
## G*Rj
def get_2(df,r): # df is the data table to be calculated, r is the Rj column name (corresponding to the cost type), c is the cost column name to be calculated, and t is the cost threshold
    df['G*Rj'] = df['G'] * df[r]
    return df

# 3 Calculate Accessibility

## 3.1 Recomended Mode (With Subways)

### 3.1.1 Get Rj

#### (1) Get G*Dk

In [10]:
GD_0_t = get_1(OD, 'Time', 5400)

In [11]:
# Aggregated by IN_FID
GD_0_t_group = GD_0_t.groupby('IN_FID')['G*Dk'].sum()
GD_0_t_group = pd.DataFrame(GD_0_t_group).reset_index(drop=False)
GD_0_t_group.columns = ['IN_FID', 'Sum_G*Dk']
GD_0_t_group.head(2)

,IN_FID,Sum_G*Dk
0,0,5.433669e+06
1,1,4.874315e+06


#### (2) Get Rj

In [12]:
# join the classified and summarized G*Dk to the original df
GD_0_t_merged = pd.merge(GD_0_t_group, GD_0_t[['IN_FID', '床位数']], on='IN_FID', how='left')
## Delete duplicates
GD_0_t_merged = GD_0_t_merged.drop_duplicates().reset_index(drop=True)

In [13]:
GD_0_t_merged.sample(5)

,IN_FID,Sum_G*Dk,床位数
105,108,6.849864e+06,13000
46,47,4.729287e+05,400000
116,119,5.715605e+06,188000
71,73,5.250162e+05,700000
40,41,3.225205e+06,661000


In [14]:
GD_0_t_merged['Rj'] = GD_0_t_merged['床位数'] / GD_0_t_merged['Sum_G*Dk']

In [15]:
GD_0_t_merged.head(5)

,IN_FID,Sum_G*Dk,床位数,Rj
0,0,5.433669e+06,990000,0.182197
1,1,4.874315e+06,990000,0.203105
2,2,2.942505e+05,990000,3.364480
3,3,3.570272e+05,713000,1.997047
4,4,4.106888e+06,713000,0.173611


In [16]:
OD_0_t_Rj = pd.merge(OD, GD_0_t_merged[['IN_FID', 'Rj']], on='IN_FID', how='left')

In [17]:
OD_0_t_Rj.head(2)

,Index,IN_FID,NEAR_FID,机构名,床位数,wgs_lng,wgs_lat,POP,lng_84,lon_84,Time,Cost,Time_5,Cost_5,G,G*Dk,Rj
0,1,0,411,广州市第一人民医院,990000,113.252797,23.133996,120239.8201,113.268876,23.133263,2042,2,1974,2,0.824632,99153.621860,0.182197
1,2,0,444,广州市第一人民医院,990000,113.252797,23.133996,100080.9571,113.249658,23.151595,1606,2,2143,2,0.890050,89077.071955,0.182197


### 3.1.2 Get Aj

In [18]:
# Get G*Rj
GR_0_t = get_2(OD_0_t_Rj, 'Rj')

In [19]:
GR_0_t.sample(2)

,Index,IN_FID,NEAR_FID,机构名,床位数,wgs_lng,wgs_lat,POP,lng_84,lon_84,Time,Cost,Time_5,Cost_5,G,G*Dk,Rj,G*Rj
13161,91658,77,46,广东省第二人民医院,433000,113.339985,23.090001,6908.763452,113.553502,22.695600,5153,10,13762,9,0.070459,486.787060,0.115691,0.008152
7135,52892,46,447,广东省中医院芳村医院,500000,113.238676,23.128581,115899.985900,113.308224,23.150748,2646,3,4710,4,0.712496,82578.306108,0.075204,0.053582


In [20]:
# Aj was obtained by 'NEAR_FID' classification
result_0_t = GR_0_t.groupby('NEAR_FID')['G*Rj'].sum()

result_0_t.head(5)

NEAR_FID
20    0.000751
24    0.355750
29    0.002090
32    0.875654
33    0.401598
Name: G*Rj, dtype: float64

In [21]:
# Convert to df
result_0_t = pd.DataFrame(result_0_t).reset_index(drop=False)
result_0_t.columns = ['NEAR_FID', 'A_0_t']
result_0_t.head(5)

,NEAR_FID,A_0_t
0,20,0.000751
1,24,0.355750
2,29,0.002090
3,32,0.875654
4,33,0.401598


-------------------------------------------------------------------------------------------------------------------------

## 3.2 No-Subway Mode

### 3.2.1 Get Rj

#### (1) Get G*Dk

In [22]:
GD_5_t = get_1(OD, 'Time_5', 22616)

In [23]:
# Aggregated by IN_FID
GD_5_t_group = GD_5_t.groupby('IN_FID')['G*Dk'].sum()
GD_5_t_group = pd.DataFrame(GD_5_t_group).reset_index(drop=False)
GD_5_t_group.columns = ['IN_FID', 'Sum_G*Dk']

#### (2) Get Rj

In [24]:
# join the aggregated G*Dk to the original df
GD_5_t_merged = pd.merge(GD_5_t_group, GD_5_t[['IN_FID', '床位数']], on='IN_FID', how='left')
## Delete duplicates
GD_5_t_merged = GD_5_t_merged.drop_duplicates().reset_index(drop=True)

In [25]:
GD_5_t_merged['Rj'] = GD_5_t_merged['床位数'] / GD_5_t_merged['Sum_G*Dk']

In [26]:
GD_5_t_merged.head(5)

,IN_FID,Sum_G*Dk,床位数,Rj
0,0,9.870961e+06,990000,0.100294
1,1,8.908461e+06,990000,0.111130
2,2,1.225795e+06,990000,0.807639
3,3,8.276107e+05,713000,0.861516
4,4,8.798226e+06,713000,0.081039


In [27]:
OD_5_t_Rj = pd.merge(OD, GD_5_t_merged[['IN_FID', 'Rj']], on='IN_FID', how='left')

### 3.2.2 Get Aj

In [28]:
# Get G*Rj
GR_5_t = get_2(OD_5_t_Rj, 'Rj')
# Aj was obtained by 'NEAR_FID' classification
result_5_t = GR_5_t.groupby('NEAR_FID')['G*Rj'].sum()

result_5_t.head(5)

NEAR_FID
20    0.006929
24    0.756982
29    0.007228
32    0.767548
33    0.758182
Name: G*Rj, dtype: float64

In [29]:
# Convet to df
result_5_t = pd.DataFrame(result_5_t).reset_index(drop=False)
result_5_t.columns = ['NEAR_FID', 'A_5_t']
result_5_t.head(5)

,NEAR_FID,A_5_t
0,20,0.006929
1,24,0.756982
2,29,0.007228
3,32,0.767548
4,33,0.758182


# 4 Organize the resulting data

In [30]:
# Merge
result = pd.merge(result_0_t, result_5_t, on='NEAR_FID')

In [31]:
rows1 = result.shape[0]
print(f"数据框的行数是: {rows1}")

数据框的行数是: 1144


In [32]:
# Check
result.head(1)

,NEAR_FID,A_0_t,A_5_t
0,20,0.000751,0.006929


In [33]:
# Merge the POP point coordinates
## Extract useful information from OD data box
OD_sub = OD[['NEAR_FID', 'lng_84', 'lon_84']]

## Delete duplicates (if not, columns will be duplicated after the left join)
OD_sub_unique = OD_sub.drop_duplicates()

## left join
result = pd.merge(result, OD_sub_unique, on='NEAR_FID', how='left')

# Calculate the difference
result['△A_t'] = result['A_0_t'] - result['A_5_t']

In [34]:
# Check
result.sample(5)

,NEAR_FID,A_0_t,A_5_t,lng_84,lon_84,△A_t
1019,1499,0.622774,0.646810,113.707001,23.559599,-0.024036
186,278,0.031741,0.245806,113.305759,23.006332,-0.214065
63,135,0.691246,1.001277,113.458495,22.823479,-0.310030
445,566,0.807433,1.126212,113.523957,23.201602,-0.318779
939,1340,0.863300,2.484195,113.666394,23.488092,-1.620895


In [35]:
rows2 = result.shape[0]
print(f"Row number: {rows2}")

Row number: 1144


# 5 Output

In [36]:
result.to_csv('Data/accessibility/acc_result/acc0813_2.csv', index=False, encoding='utf-8')